In [52]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
batch_size = 64
epochs = 50
latent_dim = 256  # latent dimension of encoding space
n_samples = 10000
# data path
data_path = 'C:\\Users\\RUPESH\\Desktop\\Datascience_new\\NLP\\Language_Translation\\fra.txt'

vectorize the data

In [53]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(n_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split('\t')
    print(target_text)
    # we use 'tab' as the 'start sequence' character
    # for the target and '\n' as 'end sequence' character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

Va !
Marche.
Bouge !
Salut !
Salut.
Cours !
Courez !
Prenez vos jambes à vos cous !
File !
Filez !
Cours !
Fuyez !
Fuyons !
Cours !
Courez !
Prenez vos jambes à vos cous !
File !
Filez !
Cours !
Fuyez !
Fuyons !
Qui ?
Ça alors !
À terre !
Baisse-toi !
Baissez-vous !
Au feu !
À l'aide !
Cache-toi.
Cachez-vous.
Saute.
Saute.
Ça suffit !
Stop !
Arrête-toi !
Attends !
Attendez !
Attendez.
Attends !
Attendez !
Attends.
Attendez.
Commencez.
Commence.
Poursuis.
Continuez.
Poursuivez.
Bonjour !
Salut !
Je comprends.
Aha.
J'essaye.
J'ai gagné !
Je l'ai emporté !
J’ai gagné.
Oh non !
Calme-toi.
Détends-toi !
Détendez-vous !
Relaxe, Max !
Cool, Raoul !
Du calme !
Relaxe !
Calmez-vous !
Détends-toi.
Détends-toi !
Du calme.
Tranquille.
Souriez !
Souris pour la caméra.
Souriez !
Pardon ?
Attaque !
Attaquez !
À l'attaque !
Achetez-la !
Achète-le !
Achetez-le !
Achète-la !
Santé !
À votre santé !
Merci !
Tchin-tchin !
Mangez-le.
Mange-le.
Lève-toi.
Lève-toi !
Debout.
Va, maintenant.
Allez-y maintenant

Vous, décidez.
C'est toi qui l'as fait !
C'est vous qui l'avez fait !
Vous pouvez partir.
Tu peux y aller.
Tu peux partir.
Tu peux t'en aller.
Vous pouvez vous en aller.
Vous pouvez y aller.
Tu as raté ton coup.
Tu as loupé ton coup.
Tu es vilain.
Tu es grand.
Tu es grande.
Vous êtes grand.
Vous êtes grande.
Vous êtes grands.
Vous êtes grandes.
T'es marrante.
T'es marrant.
Tu es bizarre.
Vous êtes bizarre.
Vous êtes bizarres.
Tu es vieux.
Tu es vieille.
Vous êtes vieux.
Vous êtes vieilles.
Vous êtes vieille.
Tu es triste.
Vous êtes triste.
Tu es timide.
Vous êtes timide.
Vous avez gagné !
Tu as gagné !
Vous êtes les vainqueurs !
Vous avez gagné !
Tu as gagné !
Vous l'avez emporté !
Tout va bien.
J'ai faim !
Suis-je idiot ?
Suis-je idiote ?
Quiconque est-il dans la maison ?
Quiconque est-il à la maison ?
Qui que ce soit est-il à la maison ?
Quiconque est-il blessé ?
Qui que ce soit est-il blessé ?
Avons-nous terminé ?
En avons-nous terminé ?
Sommes-nous perdus ?
Sommes-nous perdues ?
So

Comme c'est excitant !
Que c'est excitant !
Quelle horreur !
C'est pitoyable !
Comme c'est pathétique !
C'est pathétique !
Comme c'est romantique !
Comme c'est romantique !
Quel romantisme !
Comment ça se passe, à l'école ?
Serrez-moi fort.
Serre-moi fort.
Serre-moi bien fort.
Serrez-moi bien fort.
J'admire Tom.
Je t'admire.
Je vous admire.
Je gagne toujours.
J'ai honte.
Je suis à la maison.
Je suis confus.
Je suis confuse.
Je suis curieux.
Je suis tellement malade.
Je suis si malade.
C'est moi qui suis tellement malade.
C'est moi qui suis si malade.
Je suis en train de travailler.
J'ai présenté mes excuses.
J'ai mangé des pommes.
J'ai mangé du caviar.
J'y crois.
Je le crois.
J'ai cassé ça.
Je l'ai appelé.
Je vous ai appelés.
Je vous ai appelées.
Je vous ai appelée.
Je vous ai appelé.
Je t'ai appelé.
Je t'ai appelée.
Je suis venu seul.
Je suis venue seule.
Je peux le réparer.
Je peux la réparer.
Je peux me débrouiller.
Je ne sais pas dessiner.
Je n'y peux rien.
Je ne peux pas bouger.
J

Il est devenu aveugle.
Il est Américain.
C'est un rouspéteur.
Il est jésuite.
C'est un aîné.
C'est un ancien.
C'est un retraité.
C'est un magnat.
Il est accro.
Il a une assuétude.
Il est drogué.
Il est adorable.
Il est derrière moi.
Il est après moi.
Il est après mes fesses.
Il me poursuit.
Il est embêtant.
Il est fou.
Il est à Tokyo.
Il est innocent.
Il est ingénu.
Il est peu sûr de lui.
Il manque de confiance en lui.
Il n'est pas un saint.
Ce n'est pas un saint.
Il n'est pas à la maison.
Il n'est pas malade.
Il est indigné.
Il est impitoyable.
Il est sans pitié.
Il est si jeune.
Il est tellement jeune.
Il étudie.
Il est en train d'étudier.
Il est trop occupé.
Il est trop lent.
Il est très malade.
Il est fort malade.
Il est ton fils.
C'est ton fils.
C'est votre fils.
Il est votre fils.
Servez-vous.
Sers-toi.
Le voilà.
Le voici qui vient.
Le voici.
Sur place ou à emporter ?
Il saignait du nez.
Il avait le nez qui saignait.
Retiens la porte !
Retenez la porte !
Ne quitte pas !
Tenez la 

Puis-je y aller en premier ?
Puis-je en avoir un ?
Puis-je en avoir une ?
Puis-je t'aider ?
Puis-je vous aider ?
Puis-je t'aider ?
Puis-je me joindre à toi ?
Puis-je me joindre à vous ?
Puis-je vous embrasser ?
Puis-je voir cela ?
Puis-je voir ça ?
Puis-je voir ça ?
Puis-je voir ceci ?
Puis-je également voir ?
Puis-je m'asseoir ?
Puis-je m'asseoir ici ?
Je peux m'asseoir ici ?
Puis-je y toucher ?
Puis-je le toucher ?
Puis-je la toucher ?
Est-ce que j'ai le droit d'y toucher ?
Est-ce que Tom peut nous voir ?
Pouvons-nous demander à Tom ?
Pouvons-nous entrer ?
Est-ce qu'on peut faire ça ?
Peux-tu annuler ?
Pouvez-vous annuler ?
Sais-tu jongler ?
Savez-vous jongler ?
Peux-tu le voir ?
Pouvez-vous le voir ?
De l'argent liquide, c'est mieux.
Attrape le ballon.
Attrape la balle.
Les chats sont malins.
Le changement est bénéfique.
Vérifiez tout le monde.
Vérifie tout le monde.
Regardez ça.
Règle ça.
Vérifie ça.
Vérifie ton chapeau.
Ferme le livre.
Ferme la porte !
Fermez la porte !
Ferme le p

In [54]:
input_texts[:20]

['Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.']

In [55]:
target_texts[:20]

['\tVa !\n',
 '\tMarche.\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n']

In [56]:
len(target_texts)

10000

In [57]:
len(target_chars), len(input_chars)

(92, 71)

In [58]:
input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [59]:
print('Number of samples', len(input_texts))
print('Number of unique input tokens', num_encoder_tokens)
print('Number of unique output tokens', num_decoder_tokens)
print('Max sequence length for input', max_encoder_seq_length)
print('Max sequence length for output', max_decoder_seq_length)

Number of samples 10000
Number of unique input tokens 71
Number of unique output tokens 92
Max sequence length for input 15
Max sequence length for output 59


In [60]:
input_token_index = dict([(char, i) for i ,char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i , char in enumerate(target_chars)])

In [61]:
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype ='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype ='float32') 
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype ='float32')

In [62]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder input data by one timestamp
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0:
            #decoder target data will be ahead by one timestamp 
            # and will not include the start character
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [63]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Defining Input sequence to process

In [64]:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# discarding encoder_output and only keep states info
encoder_states = [state_h, state_c]

Set up the decoder, using encoder_states as initial state

In [65]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we set up our decoder to return full output sequences
# and to return internal state as well. we dont use the return state
# in the training model, but use them in inference
decoder = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs , _, _= decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Defining The model

Taking encoder_input_data, and decoder_input_data convert decoder_target_data

In [68]:
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
# training
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size, epochs=epochs,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 30s 4ms/sample - loss: 1.3049 - accuracy: 0.7286 - val_loss: 1.1306 - val_accuracy: 0.7029
Epoch 2/50
8000/8000 [==============================] - 32s 4ms/sample - loss: 0.8777 - accuracy: 0.7576 - val_loss: 0.9350 - val_accuracy: 0.7487
Epoch 3/50
8000/8000 [==============================] - 30s 4ms/sample - loss: 0.7525 - accuracy: 0.7963 - val_loss: 0.8009 - val_accuracy: 0.7818
Epoch 4/50
8000/8000 [==============================] - 27s 3ms/sample - loss: 0.6468 - accuracy: 0.8193 - val_loss: 0.7077 - val_accuracy: 0.7990
Epoch 5/50
8000/8000 [==============================] - 28s 3ms/sample - loss: 0.5843 - accuracy: 0.8327 - val_loss: 0.6651 - val_accuracy: 0.8090
Epoch 6/50
8000/8000 [==============================] - 28s 3ms/sample - loss: 0.6909 - accuracy: 0.8161 - val_loss: 0.6650 - val_accuracy: 0.8072
Epoch 7/50
8000/8000 [==============================] - 28s 3ms/sample

In [76]:
model.save('encoder_decoder_LSTM')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: encoder_decoder_LSTM\assets


In [72]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [73]:
# reverse_lookup token index to decode sequence back to something redable
reverse_input_char_index = dict(
                                (i, char) for char ,i in input_token_index.items())
reverse_target_char_index =dict(
                        (i,char) for char, i in target_token_index.items())

In [74]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [75]:
for seq_index in range(100):
    # Take one sequence (part of training set)
    # Trying our decoding
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('#####')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

#####
Input sentence: Go.
Decoded sentence: Va !

#####
Input sentence: Go.
Decoded sentence: Va !

#####
Input sentence: Go.
Decoded sentence: Va !

#####
Input sentence: Hi.
Decoded sentence: Salut.

#####
Input sentence: Hi.
Decoded sentence: Salut.

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run!
Decoded sentence: File !

#####
Input sentence: Run.
Decoded sentence: Fuyez !

#####
Input sentence: Run.
Decoded sentence: Fuyez !

#####
Input sentence: Run.
Decoded sentence: Fuyez !

#####
Input sentence: Run.
Decoded sentence: Fuyez !

#####
Input sentence: Run.
Decoded sentence: Fuyez !

#####
Input sentence: Run.
Decoded sentence: Fuyez !